In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.base import TransformerMixin, BaseEstimator
import string
import re

# Get Data from Website

!wget https://s3.amazonaws.com/drivendata/data/4/public/da1dd36a-a497-42c7-b3f3-4a225944bdba.zip

# Import Data

In [2]:
df = pd.read_csv('Education/TrainingData.csv', index_col=0)
df.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating,NaN,...,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,CONTRACTOR SERVICES,...,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating,Personal Services - Teachers,...,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating,EMPLOYEE BENEFITS,...,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating,TEACHER COVERAGE FOR TEACHER,...,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400277 entries, 134338 to 415831
Data columns (total 25 columns):
Function                  400277 non-null object
Use                       400277 non-null object
Sharing                   400277 non-null object
Reporting                 400277 non-null object
Student_Type              400277 non-null object
Position_Type             400277 non-null object
Object_Type               400277 non-null object
Pre_K                     400277 non-null object
Operating_Status          400277 non-null object
Object_Description        375493 non-null object
Text_2                    88217 non-null object
SubFund_Description       306855 non-null object
Job_Title_Description     292743 non-null object
Text_3                    109152 non-null object
Text_4                    53746 non-null object
Sub_Object_Description    91603 non-null object
Location_Description      162054 non-null object
FTE                       126071 non-null float64
Func

# Divide Data into Feature Matrix and Labels

In [4]:
label_cols = ['Function',
              'Object_Type',
              'Operating_Status',
              'Position_Type',
              'Pre_K',
              'Reporting',
              'Sharing',
              'Student_Type',
              'Use']

X = df.drop(label_cols, axis=1)
y = df[label_cols]

In [5]:
X.head()

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
134338,NaN,NaN,NaN,Teacher-Elementary,NaN,NaN,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,CONTRACTOR SERVICES,BOND EXPENDITURES,BUILDING FUND,(blank),Regular,NaN,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Personal Services - Teachers,NaN,NaN,TCHER 2ND GRADE,NaN,Regular Instruction,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,EMPLOYEE BENEFITS,TEACHER SUBS,GENERAL FUND,"Teacher, Short Term Sub",Regular,NaN,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,TEACHER COVERAGE FOR TEACHER,TEACHER SUBS,GENERAL FUND,"Teacher, Secondary (High)",Alternative,NaN,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


In [6]:
y.head()

,Function,Object_Type,Operating_Status,Position_Type,Pre_K,Reporting,Sharing,Student_Type,Use
134338,Teacher Compensation,NO_LABEL,PreK-12 Operating,Teacher,NO_LABEL,School,School Reported,NO_LABEL,Instruction
206341,NO_LABEL,NO_LABEL,Non-Operating,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL
326408,Teacher Compensation,Base Salary/Compensation,PreK-12 Operating,Teacher,Non PreK,School,School Reported,Unspecified,Instruction
364634,Substitute Compensation,Benefits,PreK-12 Operating,Substitute,NO_LABEL,School,School Reported,Unspecified,Instruction
47683,Substitute Compensation,Substitute Compensation,PreK-12 Operating,Teacher,NO_LABEL,School,School Reported,Unspecified,Instruction


# Solution 1: Numerical Features Only

In [7]:
num_features = [col for col in df.columns if df[col].dtype != 'object']

In [8]:
num_features

['FTE', 'Total']

In [9]:
ct = ColumnTransformer(remainder='drop',
                       transformers=[
                           ('imputer', SimpleImputer(strategy='median'), num_features)
                       ])

one_feat_model = Pipeline([
    ('transformer', ct),
    ('classifier', DecisionTreeClassifier())
])

In [10]:
one_feat_model.fit(X,y)

Pipeline(memory=None,
     steps=[('transformer', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbose=0), ['FTE', 'Total'])])), ('classifier',...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

# Create Submission

In [29]:
def create_submission(model):
    # Load test set
    X_test = pd.read_csv('education/TestData.csv', index_col=0, 
                         dtype={1: 'object', 5: 'object', 11: 'object'})
    # Predict
    y_pred = model.predict_proba(X_test)
    
    # Prepare submission DataFrame
    class_list = model.classes_
    prefix_list = y.columns
    
    data = np.concatenate(y_pred, axis=1)
    
    all_columns = []

    vfunc = np.vectorize(lambda pre, col: pre + '__' + col)

    for n in range(len(y.columns)):
        all_columns.append(vfunc(prefix_list[n], class_list[n]))

    all_columns = np.concatenate(all_columns, axis=None)
    
    submission = pd.DataFrame(data, columns=all_columns, index=X_test.index)
    
    date = pd.datetime.now().strftime(format='%Y-%m-%d_%H%M_')
    
    submission.to_csv(f'education/{date}submission.csv')

In [12]:
create_submission(one_feat_model)

# Solution 2: Numerical and Text Data

In [13]:
X.head()

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
134338,NaN,NaN,NaN,Teacher-Elementary,NaN,NaN,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,CONTRACTOR SERVICES,BOND EXPENDITURES,BUILDING FUND,(blank),Regular,NaN,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Personal Services - Teachers,NaN,NaN,TCHER 2ND GRADE,NaN,Regular Instruction,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,EMPLOYEE BENEFITS,TEACHER SUBS,GENERAL FUND,"Teacher, Short Term Sub",Regular,NaN,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,TEACHER COVERAGE FOR TEACHER,TEACHER SUBS,GENERAL FUND,"Teacher, Secondary (High)",Alternative,NaN,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


In [14]:
text_features = [col for col in X.columns if col not in num_features]

In [15]:
text_features

['Object_Description',
 'Text_2',
 'SubFund_Description',
 'Job_Title_Description',
 'Text_3',
 'Text_4',
 'Sub_Object_Description',
 'Location_Description',
 'Function_Description',
 'Facility_or_Department',
 'Position_Extra',
 'Program_Description',
 'Fund_Description',
 'Text_1']

In [24]:
class RowConcat(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        return self
    
    def clean_cell(self, x):
        x = x.strip().lower()
        x = x.replace('-', ' ')
        x = x.translate(str.maketrans('','',string.punctuation))
        x = re.sub(r'\s{2,}', ' ', x)
        return x

    def concat_row(self, row):
        row = [self.clean_cell(c) for c in row.values if c]
        row = ' '.join(row)
        return row
    
    def transform(self, X):
        return (X.fillna('')
                 .apply(lambda x: self.concat_row(x), axis=1))

In [25]:
rc = RowConcat()

In [26]:
rc.fit_transform(X[text_features].head())

134338    teacher elementary kindergarten kindergarten g...
206341    contractor services bond expenditures building...
326408    personal services teachers tcher 2nd grade reg...
364634    employee benefits teacher subs general fund te...
47683     teacher coverage for teacher teacher subs gene...
dtype: object

In [27]:
# Clean and vectorize text features
text_pipe = Pipeline([
    ('row concat', RowConcat()),
    ('vectorizer', HashingVectorizer())
])

# Impute numerical features, concat with text features
ct = ColumnTransformer(remainder='passthrough', 
                       transformers=[
                           ('imputer', SimpleImputer(strategy='median'), num_features),
                           ('text pipeline', text_pipe, text_features)
                       ])

# Pass complete, transformed feature matrix to predictor
all_feat_model = Pipeline([
    ('transformer', ct),
    ('classifier', DecisionTreeClassifier())
])

In [28]:
all_feat_model.fit(X, y);

In [32]:
create_submission(all_feat_model)

In [35]:
# Clean and vectorize text features
text_pipe = Pipeline([
    ('row concat', RowConcat()),
    ('vectorizer', HashingVectorizer())
])

# Impute numerical features, concat with text features
ct = ColumnTransformer(remainder='passthrough', 
                       transformers=[
                           ('imputer', SimpleImputer(strategy='median'), num_features),
                           ('text pipeline', text_pipe, text_features)
                       ])

# Pass complete, transformed feature matrix to predictor
all_feat_model_1 = Pipeline([
    ('transformer', ct),
    ('classifier', ExtraTreesClassifier())
])

In [36]:
all_feat_model_1.fit(X, y);

C:\Users\konra\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [37]:
create_submission(all_feat_model_1)